In [1]:
import pandas as pd
from langdetect import detect
import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [2]:
def convert_unidecode(x):
    return unidecode.unidecode(x)

In [3]:
# remove ponctuation
def remove_punct1(x):
    p = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', "'", '"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']
    return ' '.join([string for string in x.split(' ') if string not in p])

In [4]:
# remove ponctuation
def remove_punct2(x):
    import re
    return re.sub(r'[^\w]|[^\w] ', ' ', x)

In [5]:
from googletrans import Translator
translator = Translator()

In [6]:
def translation(x):
    if detect(x) != 'en':
        x = translator.translate(x).text
        return x
    else:
        return x

In [7]:
# Define stop words to clean text (job description)
stopwords_en = set(stopwords.words('english'))

stopwords_pt = set(stopwords.words('portuguese'))

stopwords_es = set(stopwords.words('spanish'))

stopwords_de = set(stopwords.words('german'))

stopwords_nl = set(stopwords.words('dutch'))

stopwords_fr = set(stopwords.words('french'))

stopwords_fi = set(stopwords.words('finnish'))

In [8]:
def remove_stop_words(x):
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_en)]) if detect(x)=='en'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_pt)]) if detect(x)=='pt'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_es)]) if detect(x)=='es'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_de)]) if detect(x)=='de'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fr)]) if detect(x)=='fr'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_nl)]) if detect(x)=='nl'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fi)])

In [ ]:
def remove_stop_words1(x):
    stop_words_concat = list(stopwords_en)+list(stopwords_es)+list(stopwords_pt)
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in stop_words_concat])

In [ ]:
def stem_snowball(x):
    if detect(x) == 'pt':
        stemmer = SnowballStemmer('portuguese')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'en':
        stemmer = SnowballStemmer('english')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'es':
        stemmer = SnowballStemmer('spanish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'de':
        stemmer = SnowballStemmer('german')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'nl':
        stemmer = SnowballStemmer('dutch')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fr':
        stemmer = SnowballStemmer('french')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fi':
        stemmer = SnowballStemmer('finnish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    else:
        return nltk.word_tokenize(x)

In [ ]:
def bigrams(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.bigrams(x))
        return output
    except:
        pass

In [ ]:
def trigram(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.trigrams(x))
        return output
    except:
        pass

In [ ]:
def token(x):
    return nltk.word_tokenize(x)

In [ ]:
df = pd.read_csv('/home/inrx/Ironhack/TA/job-search/linkedin-project/linkedin_search.csv')
print (df.shape)

(2384, 12)


In [ ]:
df = df.drop_duplicates(keep='last')
df = df.reset_index(drop=True)
df.shape

(1801, 12)

In [ ]:
df['lang'] = df['role-description'].apply(lambda x: detect(x))

df = df[df['lang'].map(df['lang'].value_counts()) > 2]

In [ ]:
df = df.replace(to_replace=r'\n', value=' ', regex=True)

In [ ]:
df['role-description'] = df['role-description'].apply(remove_punct1)
df['role-description'] = df['role-description'].apply(remove_punct2)

In [ ]:
df['role-description'] = df['role-description'].apply(lambda x: x.lower())

In [ ]:
df['role_description1'] = df['role-description'].apply(translation)

In [ ]:
df.role_description1

In [ ]:
df['role_description1'] = df['role_description1'].apply(convert_unidecode)

In [ ]:
# Clean the text according remove the stopwords according to the language of the job post
df['role_description1'] = df['role_description1'].apply(remove_stop_words)

# Clean the text according remove the stopwords regardless the language of the job post
df['role_description1'] = df['role_description1'].apply(remove_stop_words1)

In [ ]:
df_stem = df[['link', 'role_description1']].copy()

In [ ]:
df_stem['role_description1'] = df_stem['role_description1'].apply(token)

In [ ]:
df_stem['role_des_bigrams'] = df_stem['role_description1'].apply(bigrams)

In [ ]:
df_stem['role_des_trigrams'] = df_stem['role_description1'].apply(trigram)

In [ ]:
df_stem.head()

In [ ]:
%store df_stem

In [ ]:
trigrams_list = [trigram for lst in df_stem.role_des_trigrams for trigram in lst]

In [ ]:
trigrams_list = [tuple(sorted (trigram)) for trigram in trigrams_list]

In [ ]:
trigrams = {tup:trigrams_list.count(tup) for tup in trigrams_list}

#trigrams = {k: v for k, v in trigrams.items() if v > 2}

trigrams_sorted = sorted(trigrams.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
%store trigrams_list
%store trigrams 
%store trigrams_sorted

In [ ]:
bigrams_list = [bigram for lst in df_stem.role_des_bigrams for bigram in lst]

In [ ]:
bigrams = {tup:bigrams_list.count(tup) for tup in bigrams_list}


In [ ]:
#bigrams = {k: v for k, v in bigrams.items() if v > 2}

bigrams_sorted = sorted(bigrams.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
%store bigrams_list
%store bigrams
%store bigrams_sorted

In [ ]:
tokens_list = [word for lst in df_stem.role_description1 for word in lst]

tokens = {word:tokens_list.count(word) for word in tokens_list}

tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
tokens = {word:tokens_list.count(word) for word in tokens_list}

In [ ]:
tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
%store tokens_list
%store tokens
%store tokens_sorted

In [ ]:
bigrams_sorted

In [ ]:
trigrams_sorted

In [ ]:
tokens_sorted

In [ ]:
functions = [word.strip() for word in df['function'] if word !='']

In [ ]:
functions_dict = {word:functions.count(word) for word in functions}
functions_dict = sorted(functions_dict.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
import re
def separate_capitalized(lst):
    s_list = re.findall('[A-Z][^A-Z]*', lst)
    ss_list = [nlst.split(' ') for nlst in s_list]
    return (' '.join([word.title() for lst in ss_list for word in lst if word != '']))

In [ ]:
separate_capitalized(functions_dict)

In [ ]:
functions_dict

In [ ]:
df['function'][0]